In [ ]:
## Dependencies
!pip install sodapy
import pandas as pd
import numpy as np
from sodapy import Socrata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
## Load, Take a Peek
client = Socrata("data.sfgov.org", None)

# id for the Police Dpt Incident Reports Data Set
# https://dev.socrata.com/foundry/data.sfgov.org/wg3w-h783
results = client.get("wg3w-h783", limit=20000, order="incident_datetime")
results_df = pd.DataFrame.from_records(results)

results_df.describe()

,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,row_id,incident_id,incident_number,report_type_code,...,resolution,police_district,cad_number,intersection,cnn,analysis_neighborhood,supervisor_district,latitude,longitude,point
count,20000,20000,20000,20000,20000,20000,20000,20000,20000,20000,...,20000,20000,15060,18930,18930,18930,18930,18930,18930,18930
unique,8971,48,1367,1,7,14374,20000,16592,14874,4,...,4,11,10974,3775,3791,42,11,3791,3791,3791
top,2018-01-01T00:00:00.000,2018-01-01T00:00:00.000,00:00,2018,Friday,2018-02-08T19:23:00.000,103098809320,633984,180084104,II,...,Open or Active,Central,180393444,MARKET ST \ POWELL ST,34016000,Mission,6,37.784560141211806,-122.40733704162238,"{'type': 'Point', 'coordinates': [-122.4073370..."
freq,78,544,587,20000,3159,26,1,4,10,16410,...,15178,3002,26,143,143,2211,4620,143,143,143


In [ ]:
# Rounds to 5MB for 20k rows
results_df.memory_usage().sum()

20000




### Transform Idea: Lat/Long Lookup
Enhance missing values for rows that have an intersection value but no lat/long.

We first have to validate: what % of rows have a valid intersection but no corresponding lat, long, point etc.?

Though it becomes immediately apparent that no such rows exist. A dead end.

In [ ]:

mask = results_df["intersection"].notnull() & results_df["longitude"].notnull()
valid_intersection_and_longitude_count = len(results_df[mask])
valid_intersection_count = len(results_df[results_df["intersection"].notnull()])
print(valid_intersection_and_longitude_count == valid_intersection_count)
print(valid_intersection_and_longitude_count)
# Answer: 100% coverage, dead end

True
18861



### Transform Idea: Remove "Out of SF" Incidents
Because the subject of analysis is for SF-occuring incidents only.

A 20k sample reveals 3.17% of rows (not incidents, since multiple rows can represent a single incident depending on the number of associated codes).

In [ ]:
sf_only_res = results_df[
    results_df["police_district"].apply(lambda x: str(x) != "Out of SF")
]
removed_ct = len(results_df) - len(sf_only_res)
print(f"discrepancy count for 20k sample: {removed_ct}")
print(f"{removed_ct / len(results_df)*100}%")

discrepancy count for 20k sample: 634
3.17%


In [ ]:
results_df.head()

,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,row_id,incident_id,incident_number,report_type_code,...,resolution,police_district,cad_number,intersection,cnn,analysis_neighborhood,supervisor_district,latitude,longitude,point
0,2018-01-01T00:00:00.000,2018-01-01T00:00:00.000,00:00,2018,Monday,2021-05-04T18:13:00.000,103098809320,1030988,216055213,II,...,Open or Active,Southern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01T00:00:00.000,2018-01-01T00:00:00.000,00:00,2018,Monday,2021-05-28T12:57:00.000,103476209029,1034762,210329436,II,...,Open or Active,Taraval,211481637,HOLLOWAY AVE \ CORONA ST,22714000,Oceanview/Merced/Ingleside,11,37.72165880704187,-122.46761532399763,"{'type': 'Point', 'coordinates': [-122.4676153..."
2,2018-01-01T00:00:00.000,2018-01-01T00:00:00.000,00:00,2018,Monday,2021-08-10T13:10:00.000,105888471000,1058884,210508088,II,...,Open or Active,Tenderloin,212221621,EDDY ST \ JONES ST,24929000,Tenderloin,6,37.7839325760642,-122.41259527758581,"{'type': 'Point', 'coordinates': [-122.4125952..."
3,2018-01-01T00:00:00.000,2018-01-01T00:00:00.000,00:00,2018,Monday,2021-06-02T07:02:00.000,103616709029,1036167,210329436,IS,...,Open or Active,Taraval,211481637,HOLLOWAY AVE \ CORONA ST,22714000,Oceanview/Merced/Ingleside,11,37.72165880704187,-122.46761532399763,"{'type': 'Point', 'coordinates': [-122.4676153..."
4,2018-01-01T00:00:00.000,2018-01-01T00:00:00.000,00:00,2018,Monday,2021-09-09T20:51:00.000,106891109320,1068911,216126472,II,...,Open or Active,Northern,NaN,TURK ST \ WEBSTER ST,25972000,Western Addition,5,37.78071557225048,-122.43042441598713,"{'type': 'Point', 'coordinates': [-122.4304244..."


### Transform Idea: Minor vs Major incidents
The public can **only** file online reports for "minor incidents" (think vandalism, harassing phone calls, lost property).

Crime reports, or what I dub "major incidents" such as stolen vehicles, identity theft etc. **cannot** be filed online.

Thankfully we don't need to manually sift through all the possible incident codes to draw this classification. We can derive the family of codes for "minor incidents" by: 
a) filtering cases for "filed_online == True"
b) find unique values for "incident_category" x "incident_subcategory" x "incident_description"

We can take this code list to designate incidents
a) as "minor" if they only contain minor codes
b) major otherwise

This distinction opens up analysis on questions such as:
- digital adoption, what % of minor incidents have an associated coplogic report over time
- what are the distribution of resolutions for minor vs major incidents?
- what is the distribution of lifetimes for open/unresolved cases


In [ ]:
minor_incident_categories = results_df[results_df['filed_online'] == True].groupby(["incident_category","incident_subcategory","incident_description"]).size().reset_index().rename(columns={0:'count'})
inc_cat = [str(x) for x in minor_incident_categories["incident_category"]]
inc_subcat = [str(x) for x in minor_incident_categories["incident_subcategory"]]
inc_desc = [str(x) for x in minor_incident_categories["incident_description"]]
# yes it's hideous syntax. Explained here:
# https://stackoverflow.com/questions/35268817/unique-combinations-of-values-in-selected-columns-in-pandas-data-frame-and-count
temp = results_df.assign(is_minor = 
                  results_df["incident_category"].apply(lambda x: str(x) in inc_cat)
                  & results_df["incident_subcategory"].apply(lambda x: str(x) in inc_subcat)
                  & results_df["incident_description"].apply(lambda x: str(x) in inc_desc)).groupby('incident_id').is_minor.all().reset_index()
minor_case_ct = temp[
    temp["is_minor"] == True
].incident_id.count()    



print(f"Number of unique categories: {len(minor_incident_categories)}")
online_filed_case_ct = results_df[results_df["filed_online"] == True]["incident_id"].nunique()

print(f"Number of online_filed cases: {online_filed_case_ct}")
print(f"Number of minor cases: {minor_case_ct}")
print(f"Number of major cases: {len(temp) - minor_case_ct}")



Number of unique categories: 16
Number of online_filed cases: 4655
Number of minor cases: 5936
Number of major cases: 10656
